In [1]:
import tensorflow as tf
import time
import numpy as np
from qopt import *
from qopt.solver_algorithms import Solver

C:\Users\julia\Documents\qopt\qopt\matrix.py:80: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')
C:\Users\julia\.conda\envs\qopt_tensorflow\lib\site-packages\filter_functions\plotting.py:68: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')
C:\Users\julia\Documents\qopt\qopt\optimize.py:88: UserWarning: simanneal not installed. SimulatedAnnealing is not available
  warn('simanneal not installed. '
C:\Users\julia\Documents\qopt\qopt\plotting.py:55: UserWarning: Qutip not installed. plot_bloch_vector_evolution() is not available
  warn('Qutip not installed. plot_bloch_vector_evolution() is not available')


In [19]:
class TensorFlowSolver(Solver):
    """
    Todo:
    - add transfer function
    - add value function
    """
    def __init__(
            self,
            h_ctrl,
            h_drift,
            tau: np.array,
            # transfer_function: Optional[TransferFunction] = None,
            # amplitude_function: Optional[AmplitudeFunction] = None,
    ):
        super().__init__(
            h_drift=h_drift,
            h_ctrl=h_ctrl,
            tau=tau
        )

        self.n_time_steps = len(tau)
        self.tau = tf.constant(
            value=tau, dtype=tf.float32, shape=(self.n_time_steps, ),
            name='Time Steps'
        )
        self.h_ctrl = [tf.constant(h.data, dtype=tf.complex64, shape=h.shape)
                       for h in self.h_ctrl]
        self.h_ctrl = tf.stack(self.h_ctrl)
        self.h_drift = [
            tf.constant(h.data, dtype=tf.complex64, shape=h.shape) for h in self.h_drift]
        self.h_drift = tf.stack(self.h_drift)

    def set_optimization_parameters(self, y: np.array) -> None:
        self._opt_pars = y
        self._ctrl_amps = self._opt_pars

    def _create_dyn_gen(self):
        control_dynamics = tf.einsum(
            'tc,cij->tij', self._ctrl_amps, self.h_ctrl)
        # t: time
        # c: control operator
        # ij: indices on the control matrix
        hamiltonian = control_dynamics + self.h_drift
        self.dyn_gen = -1j * hamiltonian

    def _compute_propagation(self) -> None:
        self._create_dyn_gen()
        self._prop = tensor_matrix_exponentials(self.dyn_gen)

    def _compute_forward_propagation(self) -> None:
        self._compute_propagation()
        self._fwd_prop = tensor_forward_pass(self._prop, self.n_time_steps)

    def _compute_propagation_derivatives(self) -> None:
        pass


@tf.function
def tensor_matrix_exponentials(dyn_gen):
    propagators = tf.linalg.expm(
        input=dyn_gen,
        name='matrix_exponential'
    )
    return propagators


@tf.function
def tensor_forward_pass(propagators, num_t):
    propagator_list = tf.unstack(propagators, num=num_t, axis=0)

    forward_pass = [propagator_list[0]]
    for i in range(1, num_t):
        forward_pass.append(forward_pass[-1] @ propagator_list[i])
    return tf.stack(forward_pass)

In [20]:
n_time_steps = int(1e2)
delta_t = np.pi / n_time_steps

solver = TensorFlowSolver(
    h_ctrl=[.5 * DenseOperator.pauli_x()],
    h_drift=[0 * DenseOperator.pauli_x()] * n_time_steps,
    tau=delta_t * np.ones(n_time_steps)
)

In [21]:
solver.set_optimization_parameters(np.ones((1, 1)))
solver._create_dyn_gen()


In [22]:
tensor_matrix_exponentials(solver.dyn_gen)

<tf.Tensor: shape=(100, 2, 2), dtype=complex64, numpy=
array([[[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0.       -0.47942552j],
        [0.       -0.47942552j, 0.8775825+0.j        ]],

       [[0.8775825+0.j        , 0

In [23]:
solver._compute_forward_propagation()


In [24]:
solver.forward_propagators


<tf.Tensor: shape=(100, 2, 2), dtype=complex64, numpy=
array([[[ 0.8775825 +0.j        ,  0.        -0.47942552j],
        [ 0.        -0.47942552j,  0.8775825 +0.j        ]],

       [[ 0.54030216+0.j        ,  0.        -0.8414709j ],
        [ 0.        -0.8414709j ,  0.54030216+0.j        ]],

       [[ 0.07073709+0.j        ,  0.        -0.99749476j],
        [ 0.        -0.99749476j,  0.07073709+0.j        ]],

       [[-0.4161468 +0.j        ,  0.        -0.9092971j ],
        [ 0.        -0.9092971j , -0.4161468 +0.j        ]],

       [[-0.8011434 +0.j        ,  0.        -0.5984718j ],
        [ 0.        -0.5984718j , -0.8011434 +0.j        ]],

       [[-0.98999214+0.j        ,  0.        -0.14111981j],
        [ 0.        -0.14111981j, -0.98999214+0.j        ]],

       [[-0.9364562 +0.j        ,  0.        +0.35078323j],
        [ 0.        +0.35078323j, -0.9364562 +0.j        ]],

       [[-0.65364313+0.j        ,  0.        +0.7568022j ],
        [ 0.        +0.7568022j

In [ ]:
start = time.time()
solver._jit_compute_propagation()
end = time.time()
print(end- start)

In [ ]:


start = time.time()
solver._compute_propagation()
end = time.time()
print(end- start)